In [1]:
import sys, os
from glob import glob

sys.path.append('..')

from tqdm import tqdm
from argparse import ArgumentParser
from easydict import EasyDict
from addict import Dict

from toolz import *
from toolz.curried import *
from itertools import islice, product

import numpy as np
import torch
import torchvision
import torch.nn.functional as F

from data.dataLoader import MakeDataLoader

from model.FFTNet import FFTNet
from model.FFTNet import FFTNet_DSNT
from model.UNet import UNet
from model.DeepLab import DeepLab 
from model.TRANS import TRANS 

from utils import GET_OPTIMIZER
from torchmetrics.functional import dice_score 

from visualisation.Visualization2 import visualisation
from scipy.spatial.distance import directed_hausdorff

import skimage

import matplotlib
import matplotlib.pyplot as plt

In [2]:
def evaluate(net, dataloader, vis, config) :
    
    dice = 0
    haus = 0
    net.eval()        
    for (x, y, _) in tqdm(dataloader):

        _, logit = validStep(x.type(torch.float32).cuda(),
                             y.type(torch.float32).cuda(),
                             net)

        logit  = logit.detach().cpu().squeeze()
        target = y.detach().cpu().squeeze()

        if "FFT" in config.model :                
            pred = vis.draw_polygon(vis.get_contour(vis.pad_FFTs(logit))).type(torch.long)
            mask = vis.draw_polygon(vis.get_contour(target)).type(torch.long)

        else :                
            pred = logit.argmax(0).type(torch.long)
            mask = target.type(torch.long)

        metrics = getMetric(pred, mask)

        dice += metrics['dice']
        haus += metrics['haus']

    dice = round(dice / len(dataloader), 3)
    haus = round(haus / len(dataloader), 3)
        
    return dice, haus
        
        
@torch.no_grad()
def validStep(x, y, net):
    
    outputs = net(x)
    
    loss = net.getLoss(outputs, y)
    
    return loss, outputs['logit']
    
def getMetric(pred, mask):
    
    dice_score = lambda pred, mask : torch.sum(pred[mask==1])*2.0 / (torch.sum(pred) + torch.sum(mask))     
    
    dice = np.array(dice_score(pred.view(-1), mask.view(-1)))

    mask_ind = np.nonzero(mask.detach().cpu().numpy())
    mask_ind = np.stack(mask_ind, axis=1)

    pred_ind = np.nonzero(pred.detach().cpu().numpy())
    pred_ind = np.stack(pred_ind, axis=1)

    if len(mask_ind) == 0 or len(pred_ind) == 0:
        if len(pred_ind) == 0 and len(pred_ind) == 0:
            hausdorff = 0
        else:
            hausdorff = 256*np.sqrt(2)
    else:
        hausdorff_m2p, hausdorff_p2m = directed_hausdorff(mask_ind, pred_ind)[0], directed_hausdorff(pred_ind, mask_ind)[0]
        hausdorff = max(hausdorff_m2p, hausdorff_p2m)

    return {'dice' : dice,
            'haus' : hausdorff}

In [3]:
def parse(task, model, modelName, encoderName, fold):

    parser = ArgumentParser()

    parser.add_argument("--dataPath", type=str, default="../data/datasets")
    parser.add_argument("--logPath", type=str, default="../log")
    parser.add_argument("--ckptPath", type=str, default="../ckpt")
    parser.add_argument("--visPath", type=str, default="../vis")
    
    # training related    
    parser.add_argument("--augType", type=str, default="aug0", help = "aug0 | aug1")
    
    # hyprer parameter
    parser.add_argument("--inputSize", type=int, default=256)    
    parser.add_argument("--batchN", type=int, default=8)
    parser.add_argument("--epochN", type=int, default=500)
    parser.add_argument("--optimizer", type=str, default="Adam")
    parser.add_argument("--lr", type=float, default= 3e-4)
    parser.add_argument("--weight_decay", type=float, default= 0)
    
    #hardware
    parser.add_argument("--cpuN", type=int, default=4)
    parser.add_argument("--gpuN", type=int, default=1, help = "0|1|2|3")

    config = first(parser.parse_known_args())
    
    config.task       = task
    config.model      = model
    config.modelName  = modelName
    config.encoderName = encoderName
    config.fold       = fold
    
    # pick a gpu that has the largest space
    os.environ["CUDA_DEVICE_ORDER"]    = "PCI_BUS_ID"
    os.environ['CUDA_LAUNCH_BLOCKING'] = "3"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(config.gpuN)
    
    if config.model == "FFTNet":        
        config.learnSize   = 21
        config.codeSize    = 71        
        net = FFTNet.NETS(config.encoderName, config.inputSize, config.learnSize)

    if config.model == "FFTDSNTNet":        
        config.learnSize   = 21
        config.codeSize    = 71
        config.cov         = 0.1
        net = FFTNet_DSNT.NETS(config.encoderName, config.inputSize, config.learnSize, cov = config.cov)        
        
    if config.model == "UNet":        
        net = UNet.NETS(modelName = config.modelName)
        
    if "DeepLab" in config.model:
        net = DeepLab.NETS(modelName = config.modelName, encoderName = "resnet50", outputStride = 8)
        
    if config.model == "TRANS":
        net = TRANS.NETS()                
        
    return config, net

In [4]:
# FOR UNet and DeepLab and TRANS
################################################################################
baselinesISIC     = compose(list, product)(["ISIC"],
                                           ["UNet", "DeepLab", "DeepLab_lovasz", "TRANS"],
                                           ["standard", "plus"], 
                                           ["ignore"],
                                           ["0", "1", "2", "3", "4"])

baselinesDISC     = compose(list, product)(["RIM_DISC"],
                                           ["UNet", "DeepLab", "DeepLab_lovasz", "TRANS"],
                                           ["standard", "plus"], 
                                           ["ignore"],
                                           ["0"])

baselinesCUP      = compose(list, product)(["RIM_CUP"],
                                           ["UNet", "DeepLab", "DeepLab_lovasz", "TRANS"],
                                           ["standard", "plus"], 
                                           ["ignore"],
                                           ["0"])

baselinesFETAL    = compose(list, product)(["FETAL"],
                                           ["UNet", "DeepLab", "DeepLab_lovasz", "TRANS"],
                                           ["standard", "plus"], 
                                           ["ignore"],
                                           ["0", "1", "2", "3", "4"])

baselinesPROSTATE = compose(list, product)(["PROSTATE"],
                                           ["UNet", "DeepLab","DeepLab_lovasz", "TRANS"],
                                           ["standard", "plus"], 
                                           ["ignore"],
                                           ["0", "1", "2", "3", "4"])

# FOR FFTDSNTNet
################################################################################
proposalsISIC     = compose(list, product)(["ISIC"],
                                           ["FFTDSNTNet"],
                                           ["ignore"],
                                           ["ResNet", "DResNet50", "DResNet26", "DResNet105"],
                                           ["0", "1", "2", "3", "4"])

proposalsDISC     = compose(list, product)(["RIM_DISC"],
                                           ["FFTDSNTNet"],
                                           ["ignore"], 
                                           ["ResNet", "DResNet26", "DResNet50", "DResNet105"],
                                           ["0"])

proposalsCUP      = compose(list, product)(["RIM_CUP"],
                                           ["FFTDSNTNet"],
                                           ["ignore"],
                                           ["ResNet", "DResNet26", "DResNet50", "DResNet105"],
                                           ["0"])

proposalsFETAL    = compose(list, product)(["FETAL"],
                                           ["FFTDSNTNet"],
                                           ["ignore"], 
                                           ["ResNet", "DResNet26", "DResNet50", "DResNet105"],
                                           ["0", "1", "2", "3", "4"])

proposalsPROSTATE = compose(list, product)(["PROSTATE"],
                                           ["FFTDSNTNet"],
                                           ["ignore"], 
                                           ["ResNet", "DResNet26", "DResNet50", "DResNet105"],
                                           ["0", "1", "2", "3", "4"])

# merge the two
################################################################################
instances = compose(list, concat)([proposalsISIC, proposalsDISC, proposalsCUP, proposalsFETAL, proposalsPROSTATE,
                                   baselinesISIC, baselinesDISC, baselinesCUP, baselinesFETAL, baselinesPROSTATE])

In [13]:
configs  = Dict()
outcomes = Dict()
# loadConfig & model
for task, model, modelName, encoderName, fold  in instances:
    
    if model == "TRANS":
        modelName   = "ignore"
        encoderName = "ignore"

    if ("lovasz" in model) and (modelName == "standard") :        
        pass
    
    else :
    
        print(5*"#" + "(" + ",".join({task, model, modelName, encoderName, fold}) + ")" + "#"*5)

        path = f'outcomes/{task}/{model}/{modelName}/{encoderName}/{fold}'
        
        if os.path.exists(path) :

            outcome = np.load(f'{path}/outcome.npy', allow_pickle='TRUE')
            outcomes[task][model][modelName][encoderName][fold] = outcome

        else:

            config, net = parse(task, model, modelName, encoderName, fold)

            if "FFT" in config.model:
                ckpt = torch.load(f"{config.ckptPath}/{config.task}/{config.model}/{config.encoderName}/{config.fold}/ckpt.pt")
            elif "TRANS" in config.model :
                ckpt = torch.load(f"{config.ckptPath}/{config.task}/{config.model}/{config.modelName}/{config.fold}/ckpt.pt")
            else:
                ckpt = torch.load(f"{config.ckptPath}/{config.task}/{config.model}/{config.modelName}/{config.fold}/ckpt.pt")
                

            net.load_state_dict(ckpt); net.eval(); net.cuda()

            vis = visualisation(21, 71, config.inputSize)

            dataLoader = MakeDataLoader(f"{config.dataPath}/{config.task}/processed",
                                        inputSize = config.inputSize,
                                        task      = config.task,
                                        mode      = "valid",
                                        fold      = int(config.fold),
                                        maskType  = ("fourier", config.codeSize) if "FFT" in config.model else "original",
                                        batchN    = 1, 
                                        cpuN      = config.cpuN,
                                        augType   = config.augType)

            if config.model == "FFTDSNTNet" :

                temp = MakeDataLoader(f"{config.dataPath}/{config.task}/processed",
                                      inputSize = config.inputSize,
                                      task      = config.task,
                                      mode      = "valid",
                                      fold      = int(config.fold),
                                      maskType  = ("fourier", config.codeSize) if "FFT" in config.model else "original",
                                      batchN    = 1, 
                                      cpuN      = config.cpuN,
                                      augType   = config.augType)

                net.initialize_Scale(temp)        
                #print(net.DSNT_Scale)

            outcome = evaluate(net, dataLoader, vis, config)

            outcomes[task][model][modelName][encoderName][fold] = outcome

            # save the results
            if not os.path.exists(path): os.makedirs(path)            
            np.save(f'{path}/outcome.npy', outcome)        

        print(outcome)

#####(FFTDSNTNet,ignore,ResNet,0,ISIC)#####
[ 0.883 18.857]
#####(FFTDSNTNet,ignore,ResNet,1,ISIC)#####
[ 0.875 21.137]
#####(FFTDSNTNet,ignore,ResNet,ISIC,2)#####
[ 0.875 21.137]
#####(FFTDSNTNet,ignore,ResNet,ISIC,3)#####
[ 0.877 20.296]
#####(FFTDSNTNet,ignore,ResNet,ISIC,4)#####
[ 0.88  19.625]
#####(FFTDSNTNet,ignore,DResNet50,0,ISIC)#####
{'dice': 0.885, 'haus': 18.876}
#####(FFTDSNTNet,ignore,DResNet50,1,ISIC)#####
{'dice': 0.877, 'haus': 21.309}
#####(FFTDSNTNet,ignore,DResNet50,ISIC,2)#####
{'dice': 0.883, 'haus': 19.535}
#####(FFTDSNTNet,ignore,DResNet50,ISIC,3)#####
{'dice': 0.877, 'haus': 19.899}
#####(FFTDSNTNet,ignore,DResNet50,ISIC,4)#####
{'dice': 0.883, 'haus': 19.059}
#####(FFTDSNTNet,ignore,DResNet26,0,ISIC)#####
{'dice': 0.881, 'haus': 19.062}
#####(FFTDSNTNet,ignore,DResNet26,1,ISIC)#####
{'dice': 0.872, 'haus': 21.648}
#####(FFTDSNTNet,ignore,DResNet26,ISIC,2)#####
{'dice': 0.877, 'haus': 20.543}
#####(FFTDSNTNet,ignore,DResNet26,ISIC,3)#####
{'dice': 0.877, 'haus

100%|██████████| 199/199 [01:18<00:00,  2.53it/s]


(0.967, 6.249)
#####(FFTDSNTNet,ignore,FETAL,ResNet,1)#####


100%|██████████| 199/199 [01:22<00:00,  2.43it/s]


(0.962, 7.511)
#####(FFTDSNTNet,ignore,FETAL,ResNet,2)#####


...

(0.827, 12.807)
#####(ignore,PROSTATE,3,TRANS)#####


100%|██████████| 91/91 [00:05<00:00, 16.23it/s]


(0.766, 19.375)
#####(ignore,PROSTATE,TRANS,4)#####


100%|██████████| 120/120 [00:06<00:00, 17.62it/s]

(0.832, 10.06)
#####(ignore,PROSTATE,0,TRANS)#####
[ 0.812 14.995]
#####(ignore,PROSTATE,1,TRANS)#####
[ 0.801 12.647]
#####(ignore,PROSTATE,TRANS,2)#####
[ 0.827 12.807]
#####(ignore,PROSTATE,3,TRANS)#####
[ 0.766 19.375]
#####(ignore,PROSTATE,TRANS,4)#####
[ 0.832 10.06 ]


In [32]:
aggFFT = Dict()

for task in ["PROSTATE", "FETAL"]:
    for model in ["FFTDSNTNet"]:
        for encoderName in ["ResNet", "DResNet26", "DResNet50", "DResNet105"]:
            
            mean = lambda axis : lambda xs : np.around(np.mean(xs, axis = axis), 2)
            std  = lambda axis : lambda xs : np.around(np.std(xs, axis = axis), 2)
            
            if task == "ISIC":
                
                try:
                    dice, haus = map(np.array, zip(*[outcomes[task][model]["ignore"][encoderName][str(fold)].item().values() for fold in range(5)]))
                except:
                    dice, haus = map(np.array, zip(*[outcomes[task][model]["ignore"][encoderName][str(fold)] for fold in range(5)]))
                    
                aggFFT[task][model][encoderName]["haus"] = str(mean(None)(haus)) + "\pm" + str(std(None)(haus))
                aggFFT[task][model][encoderName]["dice"] = str(mean(None)(dice)) + "\pm" + str(std(None)(dice))
                
            
            else:                
                
                dice, haus = outcomes[task][model]["ignore"][encoderName]["0"]
            
                aggFFT[task][model][encoderName]["haus"] = haus         
                aggFFT[task][model][encoderName]["dice"] = dice
                
                            
aggED = Dict()

for task in ["PROSTATE", "FETAL"]:
    for model in ["UNet", "DeepLab", "DeepLab_lovasz", "TRANS"]:
        for moddelName in ["standard", "plus"]:
            
            if model == "TRANS":
                moddelName = "ignore"

            if model == "DeepLab_lovasz":
                moddelName = "plus"

            mean = lambda axis : lambda xs : np.around(np.mean(xs, axis = axis), 2)
            std  = lambda axis : lambda xs : np.around(np.std(xs, axis = axis), 2)

            if task == "ISIC":
                try:
                    dice, haus = map(np.array, zip(*[outcomes[task][model][moddelName]["ignore"][str(fold)].item().values() for fold in range(5)]))
                except:
                    dice, haus = map(np.array, zip(*[outcomes[task][model][moddelName]["ignore"][str(fold)] for fold in range(5)]))

                aggED[task][model][moddelName]["haus"] = str(mean(None)(haus)) + "\pm" + str(std(None)(haus))
                aggED[task][model][moddelName]["dice"] = str(mean(None)(dice)) + "\pm" + str(std(None)(dice))

            else:                

                dice, haus = outcomes[task][model][moddelName]["ignore"]["0"]

                aggED[task][model][moddelName]["haus"] = haus
                aggED[task][model][moddelName]["dice"] = dice


In [33]:
from prettyformatter import pprint

In [34]:
pprint(aggFFT)

{
    "PROSTATE":
        {
            "FFTDSNTNet":
                {
                    "ResNet"    : {"haus": 11.529, "dice": 0.81},
                    "DResNet26" : {"haus": 10.708, "dice": 0.827},
                    "DResNet50" : {"haus": 10.836, "dice": 0.822},
                    "DResNet105": {"haus": 10.274, "dice": 0.829},
                },
        },
    "FETAL":
        {
            "FFTDSNTNet":
                {
                    "ResNet"    : {"haus": 6.249, "dice": 0.967},
                    "DResNet26" : {"haus": 5.758, "dice": 0.97},
                    "DResNet50" : {"haus": 6.48, "dice": 0.966},
                    "DResNet105": {"haus": 5.771, "dice": 0.971},
                },
        },
}


In [35]:
pprint(aggED)

{
    "PROSTATE":
        {
            "UNet":
                {"standard": {"haus": 9.907, "dice": 0.832}, "plus": {"haus": 12.528, "dice": 0.828}},
            "DeepLab":
                {"standard": {"haus": 9.778, "dice": 0.84}, "plus": {"haus": 9.225, "dice": 0.838}},
            "DeepLab_lovasz": {"plus": {"haus": 11.678, "dice": 0.833}},
            "TRANS"     : {"ignore": {"haus": 14.995, "dice": 0.812}},
        },
    "FETAL":
        {
            "UNet":
                {"standard": {"haus": 8.949, "dice": 0.972}, "plus": {"haus": 9.385, "dice": 0.973}},
            "DeepLab":
                {"standard": {"haus": 5.917, "dice": 0.977}, "plus": {"haus": 6.131, "dice": 0.976}},
            "DeepLab_lovasz": {"plus": {"haus": 5.612, "dice": 0.978}},
            "TRANS"     : {"ignore": {"haus": 6.662, "dice": 0.975}},
        },
}
